In [1]:
import time
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('ggplot')
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Here we are importing the h5 file and create a dataframe.

In [2]:
file = h5py.File('Data_Fuchs_v_2.7_Wright_Pat_Narrow_Range_energy_limit_0.01_deviation_0.1_lambda_um_0.8_points_100000_seed_6991.h5','r')
file_dict = dict()
for i in file.keys():
    file_dict[i] = file[i][:]
file.close()
physics = pd.DataFrame(file_dict)

# Here we specify the input and output variables

In [3]:
outputs = ['Max_Proton_Energy_(MeV)', 'Total_Proton_Energy_(MeV)', 'Avg_Proton_Energy_(MeV)']

In [4]:
# this is to avoid copying a pasting
inputs = []
for i in physics.columns:
    if i in outputs:
        continue
    else:
        inputs.append(i)
inputs

['Intensity_(W_cm2)',
 'Pulse_Duration_(fs)',
 'Spot_Size_(FWHM um)',
 'Target_Thickness (um)']

##### For now, we will work with a dataframe that only has Max Proton Energy as the output variable and name it df.

In [5]:
df = physics.copy()
df = df.drop(columns= outputs[1:])
df.columns

Index(['Intensity_(W_cm2)', 'Max_Proton_Energy_(MeV)', 'Pulse_Duration_(fs)',
       'Spot_Size_(FWHM um)', 'Target_Thickness (um)'],
      dtype='object')

##### The rows that contain negative Max Proton Energy will be dropped.

In [6]:
df = df[df[outputs[0]] > 0]

In [7]:
# df.to_csv('raw_data.csv', index=False)

##### The train and split sets will also be created here for df.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[[inputs[0], inputs[3]]], df[outputs[0]], test_size=0.33, random_state=6991)

##### Normalizing the data

In [ ]:
X_train_means = X_train.mean()
X_train_stds = X_train.std(ddof=1)
X_train_norm = (X_train - X_train_means) / X_train_stds

y_train_means = y_train.mean()
y_train_stds = y_train.std(ddof=1)
y_train_norm = (y_train - y_train_means) / y_train_stds

X_test_means = X_test.mean()
X_test_stds = X_test.std(ddof=1)
X_test_norm = (X_test - X_test_means) / X_test_stds

y_test_means = y_test.mean()
y_test_stds = y_test.std(ddof=1)
y_test_norm = (y_test - y_test_means) / y_test_stds

In [ ]:
X_train_norm.head()

# Scatter plot of the data to be modeled.

In [ ]:
fig1 = plt.figure(figsize=(9, 9))
ax = fig1.add_subplot(projection='3d')
ax.scatter(X_train_norm[inputs[0]],
           X_train_norm[inputs[3]],
           y_train,
           c = np.sqrt(y_train),
           label = "Data")
ax.set_title('Scatter Plot of Data')
ax.set_xlabel(f'{inputs[0]}', fontweight ='bold')
ax.set_ylabel(f'{inputs[3]}', fontweight ='bold')
ax.set_zlabel(f'{outputs[0]}', fontweight ='bold')
ax.view_init(20, 50)
plt.legend(loc="upper right")
plt.show()

# Support Vector Regression model (SVR)

##### First we create a model to predict the values of the X test set to see what the predictions look like.

In [ ]:
start_time = time.process_time()

In [ ]:
svr_poly = SVR(kernel="poly",
               C=51.2,
               gamma="scale",
               degree=3,
               epsilon=0.088,
               coef0=.082
               )
# svr_poly = LinearSVR(epsilon=0.45,
#                      tol=0.0001,
#                      C=1.0,
#                      loss='epsilon_insensitive',
#                      fit_intercept=True,
#                      intercept_scaling=1.0,
#                      dual=True,
#                      verbose=0,
#                      random_state=None,
#                      max_iter=5000
#                      )

In [ ]:
svr_poly.fit(X_train_norm[[inputs[0], inputs[3]]], np.log(y_train))

In [ ]:
my_pred = np.exp(svr_poly.predict(X_train_norm))

##### CPU time to run, train, and predict data

In [ ]:
end_time = time.process_time()
total_time = end_time - start_time
print(f'The CPU time for this model to run is {total_time} seconds.')

In [ ]:
fig2 = plt.figure(figsize=(10, 10))
ax = fig2.add_subplot(projection='3d')
ax.scatter(X_train[inputs[0]],
           X_train[inputs[3]],
           y_train,
           c = np.sqrt(y_train),
           alpha = .8,
           label= 'Data')
ax.scatter(X_train[inputs[0]],
           X_train[inputs[3]],
           my_pred,
           c = 'red',
           alpha= .5,
           label= 'Model')
ax.set_title('SVR Model of Max Proton Energy')
ax.set_xlabel(f'{inputs[0]}', fontweight ='bold')
ax.set_ylabel(f'{inputs[3]}', fontweight ='bold')
ax.set_zlabel(f'{outputs[0]}', fontweight ='bold')
ax.view_init(20, 50)
plt.legend(loc="upper right")
ax = plt.gca()
leg = ax.get_legend()
leg.legendHandles[1].set_alpha(1)
leg.legendHandles[0].set_alpha(1)
plt.show()

##### An error dataframe will be created for the log transformed data model for the testing sets. This dataframe will also contain the predicted values of the log model, and it will contain the predicted values of the model converted back into its original non log format.

In [ ]:
error_3d = X_test_norm.copy()
error_3d['Predictions'] = np.exp(svr_poly.predict(X_test_norm))
error_3d[outputs[0]] = y_test
error_3d['Relative Error'] = np.abs((y_test - error_3d['Predictions']) / y_test) * 100

In [ ]:
fig3 = plt.figure(figsize=(10, 10))
ax = fig3.add_subplot(projection='3d')
ax.scatter(X_test_norm[inputs[0]],
           X_test_norm[inputs[3]],
           error_3d['Relative Error'],
           c = np.sqrt(error_3d['Relative Error']))
ax.set_title('Relative Error of SVR Model with Max Proton Energy')
ax.set_xlabel(f'{inputs[0]}', fontweight ='bold')
ax.set_ylabel(f'{inputs[3]}', fontweight ='bold')
ax.set_zlabel(f'Relative Error %', fontweight ='bold')
ax.view_init(20, 50)
plt.show()

# Removing the mild and extreme outliers

In [ ]:
error_3d_2 = error_3d.copy()
q1 = np.percentile(error_3d_2['Relative Error'], 25)
q3 = np.percentile(error_3d_2['Relative Error'], 75)
iqr = q3 - q1
error_3d_2 = error_3d_2[(error_3d_2['Relative Error'] >= q1 - 1.5 * iqr) & (error_3d_2['Relative Error'] <= q1 + 1.5 * iqr)]

In [ ]:
error_3d_2

In [ ]:
fig5 = plt.figure(figsize=(10, 10))
ax = fig5.add_subplot(projection='3d')
ax.scatter(error_3d_2[inputs[0]],
           error_3d_2[inputs[3]],
           error_3d_2['Relative Error'],
           alpha = .8,
           c = error_3d_2['Relative Error'])
ax.set_title('Relative Error of Spline Model with Max Proton Energy Test Set')
ax.set_xlabel(f'{inputs[0]}', fontweight ='bold')
ax.set_ylabel(f'{inputs[3]}', fontweight ='bold')
ax.set_zlabel(f'Relative Error %', fontweight ='bold')
ax.view_init(20, 50)
plt.show()

# Summary

In [ ]:
x = np.arange(0, 101, 1)
idk = np.zeros(101)

for i in range(len(idk)):
    idk[i] = np.percentile(error_3d['Relative Error'], x[i])


figure(figsize=(8, 8), dpi=90)
x_ticks = np.arange(0, 101, 5)
y_ticks = np.arange(0, np.max(error_3d['Relative Error']) +1, 5)
plt.xticks(x_ticks)
plt.yticks(y_ticks)
plt.xlabel('Percent of Testing Set')
plt.ylabel('Percent Error')
plt.plot(x, idk)
plt.show()

In [ ]:
print(f'Size of training set| {len(X_train)}\nSize of testing set | {len(X_test)}')

In [ ]:
mean_squared_error(y_test, error_3d['Predictions'])

In [ ]:
print(f'CPU\nHours  | {total_time // 60**2}\nMinutes| {total_time // 60}\nSeconds| {np.abs(total_time - 60 * (total_time // 60 ))}')

In [ ]:
r2_score(y_test, error_3d['Predictions'])

In [ ]:
np.percentile(error_3d['Relative Error'], 95)

# Optimizing the hyperparameters

In [ ]:
# parameters = {'kernel': ('poly', 'rbf'),
#               'C': [51.1, 51.2, 52],
#               'degree': [3],
#               'gamma': ['auto'],
#               'epsilon': [.087, .0879, .088, .0881],
#               'coef0': [.081, .082, .0821]
#               }

In [ ]:
# svr = SVR()

In [ ]:
# clf = GridSearchCV(svr,
#                    parameters,
#                    n_jobs= -1,
#                    verbose= 2,
#                    cv = 5
#                    )

In [ ]:
# y_train_log = np.log(y_train)
# clf.fit(X_train_norm, y_train_log)

In [ ]:
# clf.best_params_

In [ ]:
# clf.best_score_

In [33]:
# clf.best_params_

In [34]:
# clf.best_score_